In [1]:
import nltk

# nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

from spacy_download import load_spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('during1_covidTweets.csv')
print(df.username.unique())
df.head()
df.count()

['AndyVermaut' 'TrendzNewsbd' 'COVIDLive' 'AhmadNorMaulana'
 'islandofcovid' 'suhail1mirza' 'yasu_yasuno_sa' 'eeeer71' 'nicktompan'
 'BenHssinAhmed2' 'Mott_Mason' 'newsfilterio' 'GlobalPandemics'
 'AquariusOnFire' 'VonPyotr' 'SharonShepshan' 'tazalot1981' 'latimes'
 'Ravie777' 'schisano85' 'morning_news24h' 'CTVNews' 'unstoppable8672'
 'Leedslad48' 'jlwoods0241' 'merapimanf' 'BobinSea' 'renztamayo'
 'pop_popculture' 'durindale' 'Hibatu3' 'boatratbown' 'CGJr_carlos'
 'yahata11' 'LouWasHe' 'andy68101744' 'mlauriat' 'DefaultNu'
 'TraderMarcoCost' 'VivekSubbiah' 'alicuff' 'meer10001' 'jonathanrockoff'
 'JerrBoi' 'Alan_Nishihara' 'NuritBaytch' 'NZStuff' 'elvisjj'
 'WatchOurCity' 'onelinernews123' 'CassieY4' 'SetTheoryVirus'
 'ZyiteGadgets' 'bmangh' 'roto_rua' 'jayjames4' 'usatodaysun21'
 'chuck85258' 'tjmakiboi' 'Su15100307' 'Melodys_Musings' 'jebender1'
 'Reuters' 'caa1000' 'DennisKendel' 'TweeterSomebody' 'Nesgirl914' 'Nan10'
 'thedextazlab' 'fox5dc' 'EGirald' 'MelanieMoore' 'easyOntario'

datetime          149172
tweet_id          149172
text              149172
username          149172
url               149172
user_location     119906
like_count        149172
retweet_count     149172
follower_count    149172
reply_count       149172
verified          149172
hashtags           28831
dtype: int64

In [9]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

#remove URL
data = [re.sub('http://\S+|https://\S+', '', sent) for sent in data]
data = [re.sub('http[s]?://\S+', '', sent) for sent in data]
data = [re.sub(r"http\S+", "", sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['US Authorizes Johnson &amp; Johnson Covid Vaccine For Emergency Use ']


In [10]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['us', 'authorizes', 'johnson', 'amp', 'johnson', 'covid', 'vaccine', 'for', 'emergency', 'use']]


In [11]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['us', 'authorizes', 'johnson', 'amp', 'johnson', 'covid', 'vaccine', 'for', 'emergency_use']


In [12]:

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Will download the model if it isn't installed yet 
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



print(data_lemmatized[:1])

[['authorize', 'emergency']]


In [15]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [16]:
vocab = len(id2word.token2id)

pd.Series([t for tokens in data_words_bigrams for t in tokens if "_" in t]).unique()

array(['fda_authorizes', 'thus_far', 'worrying_six', ...,
       'whooping_cough', 'polio_diphtheria', 'infections_ezekiel'],
      dtype=object)

In [17]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.273*"shoot" + 0.000*"ecosearch" + 0.000*"reuter" + 0.000*"campaign" + '
  '0.000*"slow" + 0.000*"allergic_reaction" + 0.000*"investigates_possible" + '
  '0.000*"frustrate" + 0.000*"near" + 0.000*"endorsement"'),
 (1,
  '0.257*"year" + 0.186*"pfizer" + 0.108*"home" + 0.082*"set" + 0.081*"last" + '
  '0.068*"child" + 0.044*"moderna" + 0.036*"effective" + 0.034*"cost" + '
  '0.028*"stay"'),
 (2,
  '0.317*"new" + 0.241*"case" + 0.237*"death" + 0.130*"die" + 0.032*"face" + '
  '0.007*"decline" + 0.006*"additional" + 0.004*"past" + 0.004*"win" + '
  '0.000*"ecosearch"'),
 (3,
  '0.816*"covid" + 0.087*"pandemic" + 0.045*"vaccination" + 0.019*"global" + '
  '0.009*"increase" + 0.007*"alert" + 0.006*"production" + 0.003*"protest" + '
  '0.000*"ecosearch" + 0.000*"world"'),
 (4,
  '0.300*"virus" + 0.202*"make" + 0.168*"spread" + 0.103*"update" + '
  '0.057*"become" + 0.039*"holiday" + 0.029*"suggest" + 0.026*"summer" + '
  '0.014*"clear" + 0.007*"book"'),
 (5,
  '0.521*"coronavirus" +

In [19]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)




Perplexity:  -20.594042272485687

Coherence Score:  0.28596678866434266


In [20]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\hannah.kim\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.218947  0.429152       1        1  22.824174
14    -0.167503 -0.150417       2        1  11.108930
2     -0.158472 -0.119485       3        1   9.667704
17    -0.122922 -0.057716       4        1   8.543589
5     -0.102261 -0.040007       5        1   7.123506
16    -0.108739 -0.044665       6        1   6.286850
15    -0.104638 -0.041660       7        1   5.568513
1     -0.108161 -0.044123       8        1   4.600750
10    -0.046076 -0.010944       9        1   4.064258
4     -0.058917 -0.016471      10        1   3.953370
13    -0.025266 -0.004958      11        1   3.521609
8      0.000325  0.001329      12        1   3.090034
9     -0.051527 -0.014044      13        1   2.804856
19     0.036306  0.008399      14        1   2.418959
18     0.075661  0.014025      15        1   1.613490
7      0.109856  0.017678      16        1   1.310978
6      0.178234  0.021963      17        1   1.282289
0      0.288306  0.018633      18        1   0.172269
11     0.292371  0.016655      19        1   0.021995
12     0.292371  0.016655      20        1   0.021879, topic_info=            Term           Freq          Total Category  logprob  loglift
9          covid  235412.000000  235412.000000  Default  30.0000  30.0000
39       vaccine  122390.000000  122390.000000  Default  29.0000  29.0000
194  coronavirus   46946.000000   46946.000000  Default  28.0000  28.0000
75           say   38772.000000   38772.000000  Default  27.0000  27.0000
5            new   38714.000000   38714.000000  Default  26.0000  26.0000
..           ...            ...            ...      ...      ...      ...
25          case       0.010804   29445.593406  Topic20 -10.1507  -6.3907
26      advanced       0.010804       1.899833  Topic20 -10.1507   3.2578
27        always       0.010804       1.899904  Topic20 -10.1507   3.2578
28        behind       0.010804       1.899816  Topic20 -10.1507   3.2578
29          come       0.010804    6125.274507  Topic20 -10.1507  -4.8206

[929 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
171       8  0.998325        able
90       14  0.998887         add
268       3  0.997791  additional
119       6  0.999136  administer
8        12  0.996780       admit
...     ...       ...         ...
205      13  0.999529        well
270       3  0.996916         win
759      11  0.999800        work
134       6  0.999543      worker
78        8  0.999878        year

[158 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 15, 3, 18, 6, 17, 16, 2, 11, 5, 14, 9, 10, 20, 19, 8, 7, 1, 12, 13])